In [18]:
import pandas as pd
import numpy as np

In [19]:
starling_ped = "./starling-data/final_filtering/starling_noduprel_qual_miss_filt-eop-mod__NO_HEADER.plink.ped"

In [20]:
starling_df = pd.read_csv(starling_ped, sep='\t', dtype=str, header=None)

In [21]:
indiv_1_data = list(starling_df.iloc[0])

In [22]:
from enum import IntEnum

class AlleleBase(IntEnum):
    A = 0
    T = 1
    G = 3
    C = 4
    N = 5 # missing allele

    def __str__(self):
        return self.name

class string(str):
    def toAllele(self) -> object:
        # convert string astr to AlleleBase
        self = self.strip().upper()
        if self == 'A':
            return AlleleBase.A
        if self == 'T':
            return AlleleBase.T
        if self == 'G':
            return AlleleBase.G
        if self == 'C':
            return AlleleBase.C
        if self == 'N' or self == '0' or self.upper() == "NAN":
            return AlleleBase.N
        raise Exception(f"Allele must be A, T, G, C, or N, received {self}")


class SNP():
    '''
    Will return True for missing SNPs like NN-NN or NT-NA
    '''
    def __init__(self, a1: AlleleBase, a2: AlleleBase):
        self._a1 = a1
        self._a2 = a2
        if a1 > a2:
            self._a1 = a2 
            self._a2 = a1

    def __eq__(self, other: AlleleBase) -> bool:
        if self._a1 == other._a1 and self._a2 == other._a2:
            return True
        if self._a1 == other._a2 and self._a2 == other._a1:
            # expect a1, a2, to be ordered by ctor, but check anyway, 
            # could have unexpected override
            return True
        return False

    # in Python 3, if __ne__ is undefined, it defaults to inverse of __eq__

    def __str__(self) -> str:
        return f"{str(self._a1)}{str(self._a2)}"

    def __len__(self):
        # number of alleles
        return 2

    def __getitem__(self, idx):
        if idx == 0:
            return self._a1
        if idx == 1 or idx == -1:
            return self._a2
        raise Exception("index out of bounds")

class StrictSNP(SNP):
    '''
    a SNP, but equals method excludes missing SNPs
    False for missing SNPs like NN-NN or NT-NA
    '''
    def __init__(self, snp: SNP):
        self._a1 = snp._a1
        self._a2 = snp._a2
        if self._a1 > self._a2:
            tmp = self._a1
            self._a1 = self._a2 
            self._a2 = tmp 

    def __eq__(self, other: AlleleBase) -> bool:
        AB = AlleleBase
        if self._a1 == AB.N or other._a1 == AB.N:
            # since a1, a2 are ordered, if a1 is N then they both must be
            return False
        if self._a2 == AB.N and other._a2 == AB.N:
            # both are haplotypes, so check equality
            return self._a1 == other._a1
        # other cases:
        #   - one is a haplotype and one has both SNPs
        #   - no unknown SNPs present
        return SNP(self._a1, self._a2) == SNP(other._a1, other._a2)


class Individual():
    '''
    for use explicitly with PED files
    '''
    def __init__(self, data: list):
        # this assumes a PED file
        self.family_id = data[0]
        self.individual_id = data[1]
        # list of SNPs
        self._SNPs = self.process_snps(data[6:])

    def process_snps(self, data):
        # convert list of nucleotides to SNPs
        snps = [None] * int(len(data) / 2) # instantiate mutable list
        sidx = 0
        for idx in range(0, len(data), 2): # iterate ever 2 indices
            a1 = string(data[idx]).toAllele()
            a2 = string(data[idx+1]).toAllele()
            snps[sidx] = SNP(a1, a2)
            sidx += 1
        return snps

    def __str__(self):
        return str([str(snp) for snp in self._SNPs])

    def __eq__(self, other):
        if self.family_id != other.family_id:
            return False
        if self.individual_id != other.individual_id:
            return False

        return str(self) == str(other)

    def __len__(self):
        return len(self._SNPs)

    def __getitem__(self, idx):
        return self._SNPs[idx]


class BetaDiverstiy(Individual):
    def __init__(self, indiv: Individual):
        #super().__init__()
        self.family_id = indiv.family_id
        self.individual_id = indiv.individual_id
        # list of SNPs
        self._SNPs = indiv._SNPs

    def jaccard(self, other: Individual):
        if self == other:
            return 0 # no differentiation

        def noMatchFound(snp1: StrictSNP, snp2: StrictSNP) -> bool :
            s11 = snp1[0]
            s12 = snp1[1]
            s21 = snp2[0]
            s22 = snp2[1]
            if s11 == s21 and s11 != AlleleBase.N:
                return False
            if s12 == s21 and s12 != AlleleBase.N:
                return False
            if s11 == s22 and s11 != AlleleBase.N:
                return False
            if s12 == s22 and s12 != AlleleBase.N:
                return False
            return True

        a_joint_presences = 0
        b_c_half_match = 0
        d_joint_absence = 0
        for idx in range(0, len(self)):
            snp1 = StrictSNP(self[idx])
            snp2 = StrictSNP(other[idx])
            if snp1 == snp2:
                a_joint_presences += 1
            else:
                if noMatchFound(snp1, snp2):
                    d_joint_absence += 1
                else:
                    b_c_half_match += 1

        numerator = a_joint_presences
        denominator = a_joint_presences + b_c_half_match
        result = 1 - numerator / denominator
        return result


In [23]:
len(starling_df)
indivs = [None] * len(starling_df) # create vector of individual objects

In [24]:
starling_np = starling_df.to_numpy()

In [25]:
# create a list of Individual objects
indiv_idx = 0
for row_idx in range(0, len(starling_np)):
    indiv = Individual(starling_np[row_idx])
    indivs[indiv_idx] = indiv
    indiv_idx += 1

In [26]:
# individual -> location mapping

indiv2site = {}
indiv2site["ATB"] = "ANT (BE)"
indiv2site["AUK"] = "AUK (NZ)"
indiv2site["HAM"] = "ANT (BE)"
indiv2site["BAD"] = "BLN (NZ)"
indiv2site["BRA"] = "BLN (NZ)"
indiv2site["CAN"] = "CAN (NZ)"
indiv2site["MONK"] = "MKW (UK)"
indiv2site["sv"] = "MLV (AU)"
indiv2site["ABGYY"] = "NWC (UK)"
indiv2site["BPBBv2"] = "NWC (UK)"
indiv2site["BRYY"] = "NWC (UK)"
indiv2site["BYPP"] = "NWC (UK)"
indiv2site["GYRR"] = "NWC (UK)"
indiv2site["PPYY"] = "NWC (UK)"
indiv2site["PRRR"] = "NWC (UK)"
indiv2site["PYYY"] = "NWC (UK)"
indiv2site["V10181"] = "NWC (UK)"
indiv2site["V10531"] = "NWC (UK)"
indiv2site["V10731A"] = "NWC (UK)"
indiv2site["NOR"] = "ORG (AU)"
indiv2site["PLM"] = "PLM (NZ)"
indiv2site["UHT"] = "UHT (NZ)"

In [27]:
unique_sites = [
    "ANT (BE)", "AUK (NZ)", "BLN (NZ)", 
    "CAN (NZ)", "MKW (UK)", "MLV (AU)", 
    "NWC (UK)", "ORG (AU)", "PLM (NZ)", "UHT (NZ)"
]

In [28]:
# create a nested dictionary with values for each pair of individuals 
# then loop through it later and summarize all pairs per pair of sites
nested_results = {}
compared = {}
for i in range(0, len(starling_np)):
    bd = BetaDiverstiy(indivs[i])
    for j in range(0, len(starling_np)):
        if i == j:
            continue
        indiv = indivs[j]
        key_i = indiv2site[starling_np[i][0]] # site i
        key_j = indiv2site[starling_np[j][0]] # site j
        if key_i not in nested_results:
            nested_results[key_i] = {}
        if key_j not in nested_results[key_i]:
            nested_results[key_i][key_j] = []
        nested_results[key_i][key_j].append(bd.jaccard(indiv))

In [32]:
with open("heatmap5_median.csv", "w") as f:
    f.write("site1,site2,value\n")
    for key_i in nested_results:
        for key_j in nested_results:
            if key_j not in nested_results[key_i]:
                print(f"missing: {key_i}-{key_j}")
            else:
                f.write(f"{key_i},{key_j},{np.median(nested_results[key_i][key_j])}\n")

In [33]:
with open("heatmap5_mean.csv", "w") as f:
    f.write("site1,site2,value\n")
    for key_i in nested_results:
        for key_j in nested_results:
            if key_j not in nested_results[key_i]:
                print(f"missing: {key_i}-{key_j}")
            else:
                f.write(f"{key_i},{key_j},{np.mean(nested_results[key_i][key_j])}\n")

In [31]:
# plot in R:

#library(ggplot2)
#starling_df <- read.csv("heatmap5_mean.csv")
#
#desired_order <- c("UHT (NZ)", "MLV (AU)", "PLM (NZ)", "ORG (AU)", "MKW (UK)", "CAN (NZ)", "BLN (NZ)", "AUK (NZ)", "ANT (BE)", "NWC (UK)")
##desired_order <- rev(desired_order)
#starling_df$pop1 <- factor(starling_df$site1, levels = desired_order)
#starling_df$pop2 <- factor(starling_df$site2, levels = desired_order)
#
#
#
#ggplot(starling_df, aes(x = pop1, y = pop2, fill = value, label = round(value, 3))) +
#  geom_tile() +
#  geom_text(color = "black", size = 3) +  
#  scale_fill_gradient(low = "white", high = "blue") + 
#  theme_minimal() +
#  theme(panel.grid = element_blank(), axis.title = element_blank()) +
#  coord_fixed(ratio = 1, xlim = c(0.5, length(desired_order) + 0.5), ylim = c(0.5, length(desired_order) + 0.5)) +
#  theme(axis.text.x = element_text(angle = 45, hjust = 1), axis.text.y = element_text(angle = 0, hjust = 1)) +
#  geom_tile(data = subset(starling_df, pop1 == pop2), fill = "white")